# Extract embeddings we need to save time

In [4]:
# take too long to load embedding and match it with pid in each author group, 
# preprocess the embedding speed up the process
import os
# load labeled data pid
Dataset = "pubmed"
fileDir = "Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)
pid_need_extract = []

for file in listfiles:
    if not file.startswith('.'):
        with open(fileDir+file, 'r', encoding = 'utf8') as f:
            for line in f:
                read_data = line.split("\t")
                # some record's doi contain \r or \n character in which creating issue, since we do not use those, ignore it
                if(len(read_data)==13 or len(read_data)==12):
                    pid_need_extract.append(int(read_data[0]))
                else:
                    print(len(read_data))
        f.close()
print("Total labeled records:",len(pid_need_extract))

Total labeled records: 140266


In [5]:
# extract 135796 embedding out of all embedding
pid_need_extract = list(sorted(set(pid_need_extract)))
print("Unique pid: ", len(pid_need_extract))

Unique pid:  135796


In [23]:
def read_embedding(Dataset = "pubmed", emb_type = "off"):
    emb = []
    while True:
        if emb_type == "lsa":
            modelSaveDir = "models/"+Dataset+"/lsa/text_sample=3m/"
            with open(modelSaveDir+'lsa_Matrix.pickle', "rb") as input_file:
                vec = pickle.load(input_file)
            with open(modelSaveDir+'feature_pid.pickle', "rb") as input_file:
                allPaperid = pickle.load(input_file)
            allPaperid = np.array(allPaperid)
            emb = np.column_stack((allPaperid,vec))
            break
        elif emb_type == "pv_dm":
            loadDir = "vectors/"+Dataset+"/"+emb_type+"/text_sample=3m/Doc2Vec(dmm,d100,n5,w5,mc2,s0.001,t24).txt"
            with open(loadDir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            break
        elif emb_type == "pv_dbow":
            loadDir = "vectors/"+Dataset+"/"+emb_type+"/text_sample=3m/Doc2Vec(dbow,d100,n5,mc2,s0.001,t24).txt"
            with open(loadDir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            break
        elif emb_type == "n2v":
            citation_emb_dir = "vectors/"+Dataset+"/"+emb_type+"/n2v.txt"
            with open(citation_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    if(len(read_data)==101):
                        emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            break
        elif emb_type == "node2vec":
            citation_emb_dir = "vectors/"+Dataset+"/"+emb_type+"/Word2Vec(vocab=2011361, size=100, alpha=0.025)(node2vec).txt"
            with open(citation_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split("\t")
                    embedding = read_data[1].split(" ")
                    if(len(embedding)==100):
                        pid = read_data[0]
                        emb.append([pid]+embedding)
                    else:
                        print(read_data)
            f.close()
            print("Total vector records:",len(emb))
            break
        elif emb_type == "p2v":
            paper_emb_dir = "vectors/"+Dataset+"/"+emb_type+"/p2v.txt"
            with open(paper_emb_dir, 'r', encoding = 'utf8') as f:
                for line in f:
                    read_data = line.split(" ")
                    emb.append(read_data)
            f.close()
            print("Total vector records:",len(emb))
            break
        elif emb_type == "off":
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="pv_dbow"
    return emb

In [24]:
import pickle
import numpy as np
# read pretrained embeddings
# "lsa", pv_dm", "pv_dbow", "n2v"
embedding_type = "node2vec"
embeddings = read_embedding(Dataset, embedding_type)
print(embeddings[0])

Total vector records: 2011361
['7935409', '0.54249114', '0.14928037', '-0.04716367', '-0.18850482', '-0.22684433', '0.27251586', '0.09689964', '0.30099502', '-0.34245121', '0.49871957', '0.01468165', '0.21932396', '0.62582719', '0.36555550', '-0.04816755', '0.05728608', '-0.23059747', '-0.07643029', '0.14646220', '-0.33273458', '0.31342265', '0.40353510', '0.05502405', '0.00244047', '0.29972684', '-0.04147591', '0.50466734', '-0.52230966', '-0.30938733', '0.05087136', '0.09930900', '0.05261264', '-0.09063478', '-0.43274337', '0.18963017', '-0.14184768', '0.19840829', '-0.48159114', '-0.14156160', '-0.30694669', '0.29065892', '-0.12029822', '-0.25583050', '-0.39908007', '0.22797932', '-0.11744966', '0.00758215', '0.28622550', '-0.00581321', '-0.33114481', '-0.09891855', '0.06272187', '-0.19230372', '0.03052156', '-0.42126483', '-0.09353916', '-0.11032189', '0.02874640', '-0.20783158', '0.40150747', '-0.25009573', '-0.02361267', '-0.25560263', '0.14029634', '0.41118491', '0.56567436', '0

In [25]:
print(len(pid_need_extract))
print(len(embeddings))

135796
2011361


In [30]:
def int_convertible(v):
    try:
        int(v)
        return True
    except (TypeError, ValueError):
        return False
inBaseEmbeddings = [i for i in embeddings if int_convertible(i[0])]
print(len(inBaseEmbeddings))
sorted_embeddding = sorted(inBaseEmbeddings,key=lambda x: (int(x[0])))

1565898


In [31]:
print(len(sorted_embeddding))
print(sorted_embeddding[:3])

1565898
[['1', '0.73791915', '0.09072238', '0.66387165', '-0.15642750', '-0.58963847', '0.19257537', '0.07243064', '-0.19921012', '-0.74424487', '0.30198386', '-0.48192272', '-0.85232502', '0.91994387', '0.48726916', '-0.22792912', '-0.32658714', '-0.86030722', '0.42257524', '-0.18369187', '-1.31843591', '1.33415902', '0.18075800', '0.76985914', '-0.48176253', '0.52786118', '-0.58253890', '0.98795879', '-0.62610281', '0.74185288', '-0.18666525', '-0.47149789', '0.21085562', '0.63729715', '0.70120257', '-0.23880941', '-0.11951235', '0.08186401', '0.02253628', '0.00198936', '-0.00355709', '0.25902337', '-0.03945994', '-0.51307839', '-0.14326154', '-0.44586867', '0.02154192', '0.77943206', '-0.48536626', '0.68190205', '-0.37324384', '-1.36430919', '-0.00613899', '0.68861884', '0.04359802', '-0.42069048', '-0.42228973', '-0.59585392', '-0.68803197', '-0.61496210', '0.09567051', '0.85295796', '0.26313964', '0.04832526', '0.22741793', '0.02243940', '0.27648002', '-0.34198722', '0.22065890', 

In [32]:
# extract needed embedding
wanted_embedding = []
num_fail_extract = 0
for emb in sorted_embeddding:
    if(len(pid_need_extract)==0):
        break
    while (pid_need_extract[0]<=int(emb[0])):
        if pid_need_extract[0]==int(emb[0]):
            wanted_embedding.append(emb)
            pid_need_extract.remove(int(emb[0]))
        elif (pid_need_extract[0]<int(emb[0])):
            # remove paper that not in all dataset
            print(pid_need_extract[0], " : ",emb[0])
            pid_need_extract.remove(pid_need_extract[0])
            num_fail_extract+=1
        if len(pid_need_extract)==0:
            break
print("Total paper fail to find: ",num_fail_extract)

214193  :  214198
225749  :  225752
342104  :  342333
384254  :  384256
557056  :  557156
591426  :  591564
672636  :  672924
699016  :  699040
717992  :  718133
757226  :  757232
862010  :  862032
948370  :  948511
1130673  :  1130695
1249372  :  1249404
1282013  :  1282024
1288076  :  1288325
1288203  :  1288325
1290937  :  1291045
1290938  :  1291045
1292439  :  1292612
1313286  :  1313289
1313496  :  1313515
1318185  :  1318216
1321336  :  1321337
1338878  :  1338904
1339328  :  1339348
1356900  :  1356903
1357887  :  1357903
1362215  :  1362221
1369007  :  1369008
1373959  :  1373988
1400165  :  1400512
1404718  :  1404735
1407373  :  1407388
1443098  :  1443245
1443783  :  1443824
1444132  :  1444255
1447225  :  1447227
1447752  :  1447786
1461275  :  1461279
1463214  :  1463224
1479346  :  1479352
1483592  :  1483607
1487728  :  1487736
1495578  :  1495579
1496959  :  1497014
1498843  :  1498879
1507788  :  1508026
1520785  :  1520786
1533095  :  1533152
1534239  :  1534240
1541

7932751  :  7932761
7944679  :  7944704
7945350  :  7945356
7956206  :  7956217
7961777  :  7962011
7966213  :  7966260
7972328  :  7972338
7976178  :  7976194
7989174  :  7989214
7989744  :  7989896
7996276  :  7996319
7997482  :  7997534
7999045  :  7999052
7999060  :  7999082
8003380  :  8003381
8005887  :  8005900
8006014  :  8006051
8006168  :  8006204
8009713  :  8009827
8009802  :  8009827
8010245  :  8010251
8011646  :  8011647
8026833  :  8026856
8031572  :  8031731
8042441  :  8042473
8048068  :  8048074
8051028  :  8051189
8051407  :  8051511
8057297  :  8057311
8057442  :  8057511
8057444  :  8057511
8060165  :  8060166
8060359  :  8060364
8063718  :  8063840
8073095  :  8073179
8078490  :  8078514
8079349  :  8079360
8079774  :  8079823
8086415  :  8086437
8088614  :  8088700
8088806  :  8088821
8093881  :  8093886
8095776  :  8095788
8095787  :  8095788
8098846  :  8098862
8102301  :  8102318
8108487  :  8108492
8114805  :  8114809
8115128  :  8115142
8122366  :  8122371


9580666  :  9580669
9581810  :  9581857
9586752  :  9586795
9592981  :  9593003
9594019  :  9594024
9595822  :  9595867
9596582  :  9596584
9599906  :  9599909
9602409  :  9602421
9603501  :  9603502
9607117  :  9607122
9608877  :  9608902
9609243  :  9609262
9609730  :  9609748
9614075  :  9614076
9617814  :  9617832
9622551  :  9622562
9623668  :  9623675
9625359  :  9625363
9626913  :  9626918
9628519  :  9628556
9632688  :  9632690
9633117  :  9633153
9639624  :  9639625
9641370  :  9641441
9641666  :  9641675
9646084  :  9646092
9648544  :  9648545
9650717  :  9650733
9664393  :  9664638
9664436  :  9664638
9665378  :  9665379
9665983  :  9666067
9666904  :  9666960
9678752  :  9678755
9678846  :  9678848
9688004  :  9688007
9688988  :  9689023
9689302  :  9689321
9692025  :  9692041
9698548  :  9698554
9699274  :  9699310
9701396  :  9701411
9703057  :  9703062
9703372  :  9703377
9703562  :  9703684
9704884  :  9704885
9705683  :  9705684
9706197  :  9706217
9708977  :  9708982


10970375  :  10970433
10970734  :  10970735
10972194  :  10972198
10972907  :  10972914
10978075  :  10978081
10978198  :  10978201
10981448  :  10981451
10985797  :  10985801
10991118  :  10991129
10994962  :  10994964
10995287  :  10995288
10995748  :  10995749
11000702  :  11000918
11007798  :  11007800
11009069  :  11009080
11014501  :  11014503
11014856  :  11014876
11015199  :  11015200
11020284  :  11020286
11021916  :  11021962
11023478  :  11023484
11030290  :  11030291
11031259  :  11031260
11035581  :  11035633
11036274  :  11036286
11038156  :  11038169
11038338  :  11038340
11041314  :  11041315
11041550  :  11041554
11044441  :  11044447
11044442  :  11044447
11046518  :  11047755
11048598  :  11048620
11048599  :  11048620
11048600  :  11048620
11048601  :  11048620
11052736  :  11052759
11053861  :  11053873
11055148  :  11055153
11056623  :  11056638
11059298  :  11059350
11060088  :  11060120
11063605  :  11063607
11064849  :  11064858
11069759  :  11069782
11070529  

11720491  :  11720527
11724612  :  11724621
11724661  :  11724664
11732907  :  11732910
11734040  :  11734042
11735495  :  11735510
11735522  :  11735592
11736067  :  11736169
11739864  :  11739889
11740035  :  11740164
11741083  :  11741095
11742346  :  11742355
11743737  :  11743741
11747578  :  11747606
11749325  :  11749328
11751077  :  11751104
11752818  :  11752823
11754276  :  11754312
11754598  :  11754608
11759683  :  11759685
11760172  :  11760190
11760792  :  11760805
11761075  :  11761077
11762186  :  11762330
11763455  :  11763477
11766973  :  11766977
11768641  :  11768674
11772071  :  11772077
11772868  :  11772870
11773350  :  11773392
11773650  :  11773656
11774479  :  11774801
11774485  :  11774801
11774519  :  11774801
11777979  :  11777997
11778595  :  11778618
11778816  :  11778831
11779806  :  11779826
11784920  :  11784922
11786942  :  11786943
11788282  :  11788303
11790260  :  11790279
11795596  :  11795643
11798295  :  11798371
11799265  :  11799274
11800234  

...

25927099  :  25927150
25927649  :  25927763
25927825  :  25927917
25931615  :  25931673
25932336  :  25932503
25933617  :  25933687
25934389  :  25934390
25934860  :  25934873
25936036  :  25936243
25939561  :  25939587
25939971  :  25939977
25941223  :  25941256
25941316  :  25941343
25941486  :  25941534
25942705  :  25942921
25942805  :  25942921
25942808  :  25942921
25942809  :  25942921
25944628  :  25944735
25945246  :  25945305
25945359  :  25945363
25948429  :  25948502
25948746  :  25948854
25948870  :  25949213
25948898  :  25949213
25950020  :  25950087
25951344  :  25951368
25953258  :  25953265
25954268  :  25954271
25954347  :  25954500
25954365  :  25954500
25954436  :  25954500
25954635  :  25954773
25954638  :  25954773
25954683  :  25954773
25955041  :  25955057
25955528  :  25955659
25957405  :  25957515
25958554  :  25958775
25960715  :  25960759
25960769  :  25960808
25960816  :  25960907
25960859  :  25960907
25960962  :  25961122
25961066  :  25961122
25961166  

26284327  :  26284435
26287225  :  26287387
26287229  :  26287387
26287372  :  26287387
26287744  :  26287751
26288582  :  26288653
26289096  :  26289101
26290251  :  26290327
26292219  :  26292532
26293500  :  26293568
26294656  :  26294719
26295139  :  26295242
26295467  :  26295478
26295809  :  26295915
26297136  :  26297141
26297702  :  26297731
26297752  :  26297806
26297758  :  26297806
26300087  :  26300122
26302330  :  26302331
26303409  :  26303443
26303948  :  26303991
26306039  :  26306100
26306232  :  26306502
26306275  :  26306502
26307142  :  26307144
26307380  :  26307446
26307654  :  26307825
26308261  :  26308371
26308457  :  26308473
26309826  :  26310105
26310804  :  26310835
26311725  :  26311730
26311774  :  26311778
26311776  :  26311778
26311813  :  26311867
26311851  :  26311867
26311868  :  26311874
26312428  :  26312440
26312511  :  26312557
26312879  :  26313126
26313063  :  26313126
26313729  :  26313785
26315822  :  26316008
26316596  :  26316599
26316909  

26655759  :  26655856
26656635  :  26656664
26656699  :  26656876
26657185  :  26657252
26657776  :  26657798
26658294  :  26658318
26659370  :  26659402
26659799  :  26659856
26663390  :  26663392
26664504  :  26664896
26665071  :  26665123
26665089  :  26665123
26666247  :  26666368
26668312  :  26668389
26668685  :  26668730
26670341  :  26670347
26670609  :  26670788
26671015  :  26671016
26671051  :  26671090
26671900  :  26672023
26672070  :  26672093
26672487  :  26672555
26673830  :  26673975
26674249  :  26674376
26675815  :  26675841
26676112  :  26676169
26676187  :  26676282
26676331  :  26676340
26676335  :  26676340
26676787  :  26676797
26677354  :  26677760
26678917  :  26679002
26679379  :  26679476
26680031  :  26680103
26680737  :  26680740
26680936  :  26681228
26680969  :  26681228
26681989  :  26682003
26682256  :  26682468
26682418  :  26682468
26683063  :  26683218
26683705  :  26683747
26684111  :  26684176
26684475  :  26684592
26687078  :  26687144
26691140  

26987125  :  26987311
26987548  :  26987695
26989175  :  26989198
26989176  :  26989198
26989410  :  26989425
26990062  :  26990250
26990350  :  26990636
26990384  :  26990636
26991529  :  26991640
26992832  :  26992837
26993251  :  26993285
26993777  :  26993883
26993780  :  26993883
26995455  :  26995462
26997624  :  26998311
26997989  :  26998311
26998042  :  26998311
26998518  :  26998588
26998603  :  26998662
26999691  :  26999703
27001948  :  27002000
27005629  :  27005642
27006115  :  27006226
27006468  :  27006949
27007071  :  27007133
27007245  :  27007316
27007693  :  27007719
27008495  :  27008965
27011426  :  27012185
27012775  :  27012876
27013462  :  27013499
27013674  :  27013677
27014470  :  27014630
27014938  :  27014967
27018360  :  27018388
27018728  :  27018913
27018814  :  27018913
27019697  :  27019736
27021190  :  27021425
27022420  :  27022974
27023264  :  27023279
27023672  :  27023684
27026700  :  27026931
27026865  :  27026931
27027230  :  27027285
27029656  

27549881  :  27550221
27550474  :  27550478
27556410  :  27556730
27557344  :  27557449
27560856  :  27560937
27563063  :  27563678
27571015  :  27571738
27572259  :  27572535
27572622  :  27572645
27573121  :  27573316
27573925  :  27573974
27576527  :  27577262
27578572  :  27579152
27578958  :  27579152
27581461  :  27582461
27588211  :  27588217
27589844  :  27589898
27590816  :  27591066
27597740  :  27597806
27599218  :  27599590
27600823  :  27601181
27601624  :  27602298
27605608  :  27606261
27606836  :  27607449
27606975  :  27607449
27607289  :  27607449
27609856  :  27609995
27618861  :  27618951
27618927  :  27618951
27622242  :  27622290
27623929  :  27623951
27624446  :  27624468
27635015  :  27635118
27635061  :  27635118
27640085  :  27640159
27650756  :  27651559
27653931  :  27654485
27663942  :  27664160
27668003  :  27668013
27669144  :  27669269
27672022  :  27676158
27679308  :  27679934
27681860  :  27682005
27774028  :  27774103
27799120  :  27802386
27843178  

In [33]:
print(len(wanted_embedding))

117857


In [34]:
print(wanted_embedding[:3])

[['8077', '2.10680366', '-0.18168521', '-2.67891407', '0.95571703', '-0.62763935', '-0.66293275', '7.27649069', '2.54705286', '-3.95711899', '-0.81939828', '-1.29333460', '-2.07562780', '6.73258448', '-0.02631122', '-5.71678543', '-1.99713147', '-2.90846658', '-2.78189421', '-3.38142467', '0.61138374', '4.42697573', '-0.39102498', '0.96968299', '3.01780558', '-1.36360753', '0.03772063', '0.49050260', '-3.61272645', '3.30619812', '3.92013359', '-8.00581551', '0.66971052', '-0.55340379', '1.41540527', '1.21762252', '-2.56369710', '-3.38531685', '-0.52065575', '0.51232773', '1.62535381', '1.63238883', '-1.84834182', '5.62380791', '-6.01321173', '2.46540260', '-1.34341180', '1.00722349', '0.05246630', '3.31422162', '3.52742124', '-3.29705787', '-2.34312987', '-0.02240184', '0.88363963', '-0.90571862', '-3.42181563', '1.32064342', '3.19831395', '0.21040830', '3.68332434', '4.47267103', '0.31682557', '-1.93728495', '0.98603356', '0.04244132', '1.03457618', '-2.09100103', '4.29358006', '-2.07

In [35]:
# export needed embedding to txt file
newfileDir = "vectors/"+Dataset+"/"+embedding_type+"/"
if not os.path.exists(newfileDir):
    os.makedirs(newfileDir)
    
with open(newfileDir+"extracted_labeled_"+embedding_type+".txt", "w",encoding='utf8') as f:
    for emb in wanted_embedding:
        f.write((" ".join(emb)).strip('\n')+"\n")
f.close()